# Exercise: (Dense) Matrix-Matrix Multiplication

In this exercise you will consider dense matrix-matrix multiplication

$C_{rc} = \sum_{k=1}^{n} A_{rk} B_{kc}$

for square matrices $A$, $B$, and $C$ of size $n \times n$. Despite being seemingly simple, you will see that different implementations have vastly different performance.

## Tasks

In [6]:
# loading packages
using BenchmarkTools
using CpuId
using Test
using LinearAlgebra
BLAS.set_num_threads(1)

# problem size
const N = 512 # tune this if necessary

# input (not to be modified)
const C = zeros(N, N)
const A = rand(N, N)
const B = rand(N, N);

1) Benchmark the naive implementation `matmul_naive!` below in GFLOPS (giga floating-point operations per second).

In [7]:
function matmul_naive!(C, A, B)
    @assert size(C) == size(A) == size(B)
    @assert size(C, 1) == size(C, 2)
    n = size(C, 1)
    fill!(C, zero(eltype(C)))

    # - c: for columns of B and C
    # - r: for rows    of A and C
    # - k: for columns of A and rows of B
    for c in 1:n
        for r in 1:n
            c_reg = 0.0
            for k in 1:n
                @inbounds c_reg += A[r, k] * B[k, c]
            end
            @inbounds C[r, c] = c_reg
        end
    end
    return C
end

matmul_naive! (generic function with 1 method)

**Note: Depending on the value of `N`, the test and benchmark below can take while. You might want to start with task 3 in the meantime.**

Correctness check:

In [20]:
@test matmul_naive!(C, A, B) ≈ mul!(C, A, B) # can take a while

Test Passed

Benchmark:

In [9]:
t_naive = @belapsed matmul_naive!($C, $A, $B) samples = 1 evals = 2
println("matmul_naive!: ", t_naive, " sec, performance = ", round(2.0e-9 * N^3 / t_naive, digits=2), " GFLOP/s")

matmul_naive!: 0.248197698 sec, performance = 1.08 GFLOP/s


2) Why is `matmul_naive!` so (super) slow?
    * Hints: Look at the memory access pattern of innermost loop.

**Answer:** Strided memory access for `A` in `matmul_naive!` reduces the performance. The implementation doesn't take column-major order into consideration.

3) Implement an improved version with contiguous memory access (better spatial locality) in `matmul_contiguous!` and benchmark it. How much faster is it?

In [15]:
function matmul_contiguous!(C, A, B)
    @assert size(C) == size(A) == size(B)
    @assert size(C, 1) == size(C, 2)
    n = size(C, 1)
    fill!(C, zero(eltype(C)))

    # - c: for columns of B and C
    # - r: for rows    of A and C
    # - k: for columns of A and rows of B
    
    #
    # TODO: Implement improved version with more efficient memory access / better localitly.
    #       Hint: Which loop should be the innermost?
    #
    
    # @. C = A * B

    # for c in 1:n
    #     b = B[:, c]
    #     for k in 1:n
    #         for r in 1:n
    #             @inbounds C[r, c] += (A[r, k] .* b[k])
    #         end
    #     end
    # end
    return C
end

matmul_contiguous! (generic function with 1 method)

Correctness check:

In [21]:
@test copy(matmul_contiguous!(C, A, B)) ≈ mul!(C, A, B) # this should give true, otherwise your implementation is incorrect.

# @show sum(matmul_contiguous!(C, A, B))
# @show sum(mul!(C, A, B))

Test Failed at In[21]:1
  Expression: copy(matmul_contiguous!(C, A, B)) ≈ mul!(C, A, B)
   Evaluated: [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0] ≈ [126.08438845457266 132.68806241189964 … 134.60475950448705 128.86331342247638; 123.6475168686562 131.4009019626955 … 131.6755511684123 130.22207491094935; … ; 118.98130068910869 127.5491101869948 … 131.53410731674666 124.74726116447792; 124.0260329545133 126.74451217453299 … 128.34985882266702 127.32766591972555]



LoadError: [91mThere was an error during testing[39m

In [18]:
C

512×512 Matrix{Float64}:
 126.084  132.688  120.809  126.517  …  131.504  127.175  134.605  128.863
 123.648  131.401  118.12   129.429     131.001  130.98   131.676  130.222
 124.922  130.849  123.256  126.438     137.045  124.035  132.628  128.03
 127.266  130.078  122.845  126.342     131.492  126.494  131.338  128.583
 124.927  131.26   123.788  127.229     134.051  127.206  136.425  130.598
 122.45   125.344  117.718  123.403  …  128.153  124.169  131.73   126.012
 126.875  127.568  121.207  126.616     132.813  122.583  127.645  122.984
 120.954  130.069  118.156  123.459     129.77   124.397  130.069  130.475
 119.626  126.926  119.32   124.873     132.774  124.62   128.507  127.242
 125.826  134.069  123.351  124.892     132.215  124.924  132.282  130.713
 123.883  131.006  123.115  125.868  …  131.851  124.656  129.923  127.405
 124.784  129.797  123.648  123.237     135.095  130.289  131.375  132.202
 119.559  132.135  120.952  122.769     129.896  122.814  133.178  126.307
 

Benchmark:

In [43]:
#
# TODO: Benchmark like matmul_naive! above
#
t_contiguous = @belapsed matmul_contiguous!($C, $A, $B) samples = 1 evals = 2
println("matmul_contiguous!: ", t_contiguous, " sec, performance = ", round(2.0e-9 * N^3 / t_contiguous, digits=2), " GFLOP/s")

matmul_contiguous!: 0.0003237725 sec, performance = 829.09 GFLOP/s


### Cache Blocking

The key idea of cache blocking is to perform the overall matrix-matrix multiplication in terms of smaller matrix-matrix multiplications of sub-blocks.

<img src="./imgs/dMMM_cache_blocking.png" width=800>
<br>

In [95]:
function matmul_cache_blocking!(C, A, B; col_blksize=16, row_blksize=128, k_blksize=16)
    @assert size(C) == size(A) == size(B)
    @assert size(C, 1) == size(C, 2)
    n = size(C, 1)
    fill!(C, zero(eltype(C)))

    # - c: for columns of B and C
    # - r: for rows    of A and C
    # - k: for columns of A and rows of B
    for ic in 1:col_blksize:n
        for ir in 1:row_blksize:n
            for ik in 1:k_blksize:n
                #
                # begin: cache blocking
                #
                for jc in ic:min(ic + col_blksize - 1, n)
                    for jk in ik:min(ik + k_blksize - 1, n)
                        @inbounds b = B[jk, jc]
                        for jr in ir:min(ir + row_blksize - 1, n)
                            @inbounds C[jr, jc] += A[jr, jk] * b
                        end
                    end
                end
                #
                # end: cache blocking
                #
            end
        end
    end
    return C
end

matmul_cache_blocking! (generic function with 1 method)

In [89]:
@test matmul_cache_blocking!(C, A, B) ≈ mul!(C, A, B)

Test Passed

4) Inspect the given variant `matmul_cache_blocking!` which implements cache blocking.
    * In which limit, that is, for what block size values, does the code semantically reduce to your code in 3)?
    * Staying away from these limits, how can this implementation be faster?
    
**Answer:**
* For `c_blksize = r_blksize = k_blksize = 1` and `c_blksize = r_blksize = k_blksize = n`.
* By operating on blocks that, ideally, fit into L1 or L2 cache, we maximize fast data reuse (temporal locality).

5) Benchmark the cache blocking variant. Specifically, we consider `c_blksize = 16`, `r_blksize = 128` and `k_blksize = 16` for the block sizes (default values).
    * Can you give an argument why it makes sense to choose `r_blksize` larger than the others?
    * How big (in bytes) are the blocks for `A` and `C` for these values? How does this compare to the L1 cache size of 32 KiB?
    
**Answer:**
* Julia arrays are column-major order which aligns with the row index `r`.
* Both blocks (`r_blksize * k_blksize` for A and `r_blksize * c_blksize` for C) are 16 * 128 * 8 bytes = 16 KiB big. Hence, they fit into L1 cache together.    

In [96]:
#
# TODO: Benchmark matmul_cache_blocking! like above
#
t_cache = @belapsed matmul_cache_blocking!($C, $A, $B) samples = 1 evals = 2
println("matmul_cache!: ", t_cache, " sec, performance = ", round(2.0e-9 * N^3 / t_cache, digits=2), " GFLOP/s")

matmul_cache!: 0.02180033 sec, performance = 12.31 GFLOP/s


6) Compare the performance of the cache-blocking variant to the built-in BLAS matrix multiply, i.e. `mul!(C, A, B)`.

In [97]:
#
# TODO: Benchmark mul! like above
#
t_cache = @belapsed mul!($C, $A, $B) samples = 1 evals = 2
println("matmul_cache!: ", t_cache, " sec, performance = ", round(2.0e-9 * N^3 / t_cache, digits=2), " GFLOP/s")

matmul_cache!: 0.0073317455 sec, performance = 36.61 GFLOP/s


7) **Bonus task:** vary the block sizes (in powers of 2) and see how the performance changes. (**This might take > 20 minutes.**)

In [ ]:
println("Varying block sizes:")
L1 = cachesize()[1]
for cbs in (4, 8, 16, 32, 64), kbs in (4, 8, 16, 32, 64), rbs in (4, 128, 256, 512)
    if rbs * kbs + rbs * cbs > L1 / 8
        # A block and C block don't fit into L1 cache together
        continue
    end
    t_cache_block = @belapsed matmul_cache_blocking!($C, $A, $B; col_blksize=$cbs, row_blksize=$rbs, k_blksize=$kbs) samples = 3 evals = 2
    println("matmul_cache_block ($cbs, $rbs, $kbs): ", t_cache_block, " sec, performance = ", round(2.0e-9 * N^3 / t_cache_block, digits=2), " GFLOPS\n")
end